In [70]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [71]:
hist_data = pd.read_csv('hist_data.csv')

In [72]:
orders_sort = hist_data[['buyer_id', 'pav_order_id', 'created']].drop_duplicates()\
                                                                .sort_values(by=['buyer_id', 'created'])\
                                                                .reset_index(drop=True)
buyer_last_order = orders_sort.drop_duplicates(subset=['buyer_id'], keep='last')

In [73]:
len(buyer_last_order) / len(orders_sort)

0.27632608423136606

In [74]:
train_orders = orders_sort[~np.isin(orders_sort.pav_order_id, buyer_last_order.pav_order_id)]['pav_order_id'].values
test_orders = orders_sort[np.isin(orders_sort.pav_order_id, buyer_last_order.pav_order_id)]['pav_order_id'].values

In [75]:
train_orders, test_orders = set(train_orders), set(test_orders)

In [77]:
train = hist_data[hist_data['pav_order_id'].apply(lambda x: x in train_orders)]
test = hist_data[hist_data['pav_order_id'].apply(lambda x: x in test_orders)]

In [81]:
train.shape[0] + test.shape[0], hist_data.shape[0]

(4529889, 4529889)

In [84]:
test.shape[0] / hist_data.shape[0]

0.2767352577513489

In [85]:
test = test.sort_values('pav_order_id').reset_index(drop=True)

In [93]:
orders = list(test.pav_order_id.values)
i = 0
co = orders[0]
split = []
for j in range(len(orders)):
    if orders[j] != co:
        i = 0
        co = orders[j]
    split.append(i)
    i += 1

In [101]:
test['split'] = split
totals = test.groupby('pav_order_id')['item_id'].count().reset_index()
totals.columns = ['pav_order_id', 'total']
test = test.merge(totals)
test.split = test.split / test.total

In [102]:
val_shown = test[test['split'] <= 0.7]
val_hidden = test[test['split'] > 0.7]

In [104]:
val_shown.shape, val_hidden.shape

((913558, 10), (340022, 10))

In [106]:
val_shown = val_shown.drop(['split', 'total'], axis=1)
val_hidden = val_hidden.drop(['split', 'total'], axis=1)

In [107]:
train.to_csv('train.csv', index=False)
val_shown.to_csv('val_shown.csv', index=False)
val_hidden.to_csv('val_hidden.csv', index=False)